<a href="https://colab.research.google.com/github/Daprosero/Procesamiento_Lenguaje_Natural/blob/main/1.%20Conceptos%20Preliminares/1.4.%20Representaci%C3%B3n_de_Texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo UNAL CHEC](https://www.funcionpublica.gov.co/documents/d/guest/logo-universidad-nacional)



# **Representación de Texto**
### Departamento de Ingeniería Eléctrica, Electrónica y Computación
#### Universidad Nacional de Colombia - Sede Manizales

#### Profesor: Diego A. Pérez

Vamos a comenzar cargando las librerías necesarias y el conjunto de datos. Usaremos la colección de SMS Spam, que es un conjunto de datos público con mensajes de texto etiquetados como "spam" o "ham" (no spam). Realizaremos una limpieza básica del texto para prepararlo para los siguientes pasos.

In [ ]:
!pip install  gensim

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Descargar recursos de NLTK (solo la primera vez)
nltk.download('stopwords')
nltk.download('punkt')

# Cargar los datos
url = "https://raw.githubusercontent.com/juacardonahe/Curso_NLP/refs/heads/main/data/SMSSpamCollection/SMSSpamCollection"
data = urllib.request.urlopen(url)

lines_split = [
    line.decode('latin-1').strip().split("\t")
    for line in data
]
df = pd.DataFrame(lines_split, columns=["label", "text"])

# Preprocesamiento básico del texto
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Quitar puntuación
    text = ''.join([char for char in text if char not in string.punctuation])
    # Quitar stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['cleaned_text'] = df['text'].apply(clean_text)

# Convertir etiquetas a valores numéricos (spam=1, ham=0)
df['label_num'] = df['label'].map({'spam': 1, 'ham': 0})

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,label,text,cleaned_text,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,0
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though,0


# Extracción de Características Básicas (Ingeniería Manual)
Celda de Texto

El primer enfoque, y el más intuitivo, es la ingeniería de características manuales. En lugar de analizar el significado de las palabras, extraemos metadatos o características estructurales del texto. Estas características pueden ser sorprendentemente efectivas.

Algunos ejemplos comunes son:

   * Longitud del mensaje: Los mensajes de spam a menudo son más largos o más cortos que los mensajes normales.

   * Conteo de mayúsculas: El uso excesivo de mayúsculas (¡OFERTA ESPECIAL!) es un indicador clásico de spam.

   * Conteo de números: Los mensajes de spam suelen incluir números de teléfono, precios o fechas.

   * Conteo de puntuación: Caracteres como !, $ o % pueden ser más frecuentes en el spam.

Ventajas de las características manuales:

   * Interpretabilidad: Es muy fácil entender por qué el modelo toma una decisión (ej. "el mensaje fue clasificado como spam porque tenía muchas mayúsculas").

   * Rapidez: Calcular estas características es computacionalmente muy económico.

   * No requieren un gran vocabulario: Funcionan sin necesidad de conocer todas las palabras posibles.

Vamos a crear estas características y entrenar un modelo de Regresión Logística para ver qué tan bien funcionan por sí solas.

In [ ]:
# Crear características manuales
df['message_length'] = df['text'].apply(len)
df['caps_count'] = df['text'].apply(lambda x: sum(1 for char in x if char.isupper()))
df['numbers_count'] = df['text'].apply(lambda x: sum(1 for char in x if char.isdigit()))
df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for char in x if char in string.punctuation))

# Seleccionar las características para el modelo
features = ['message_length', 'caps_count', 'numbers_count', 'punctuation_count']
X = df[features]
y = df['label_num']

# Dividir los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar y evaluar el modelo
manual_model = LogisticRegression()
manual_model.fit(X_train, y_train)
y_pred_manual = manual_model.predict(X_test)

print("Rendimiento con Características Manuales:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_manual):.4f}")
print(classification_report(y_test, y_pred_manual))

Rendimiento con Características Manuales:
Accuracy: 0.9686
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       954
           1       0.98      0.80      0.88       161

    accuracy                           0.97      1115
   macro avg       0.97      0.90      0.93      1115
weighted avg       0.97      0.97      0.97      1115



#Conversión de Texto a Valores Numéricos: One-Hot Encoding
Celda de Texto

Las características manuales son útiles, pero ignoran el contenido del mensaje. Para que un modelo de machine learning entienda las palabras, debemos convertirlas en números. La técnica más fundamental es el One-Hot Encoding.

El proceso es el siguiente:

   * Crear un vocabulario: Se recopilan todas las palabras únicas del conjunto de datos.

   * Asignar un índice: A cada palabra del vocabulario se le asigna un número entero único (un índice).

   * Crear un vector: Cada palabra se representa como un vector de ceros del tamaño del vocabulario, con un 1 en la posición de su índice.

Por ejemplo, si nuestro vocabulario es {"hola": 0, "mundo": 1, "adiós": 2}:

   * hola se convierte en [1, 0, 0]

   * mundo se convierte en [0, 1, 0]
   * adiós se convierte en [0, 0, 1]

Problemas de dimensionalidad y rareza de palabras:

   * Alta dimensionalidad: Si nuestro vocabulario tiene 10,000 palabras, cada palabra será un vector de 10,000 dimensiones. Esto es computacionalmente muy costoso.

   * Vectores dispersos (Sparsity): Los vectores están llenos de ceros, lo que es ineficiente en términos de almacenamiento y procesamiento.

   * Sin relación semántica: Los vectores de palabras como "rey" y "reina" son tan diferentes entre sí como los de "rey" y "coche", ya que todos son perpendiculares en el espacio vectorial.



In [ ]:
from sklearn.preprocessing import OneHotEncoder
sample_sentence = "go until jurong point crazy"
words = sample_sentence.split()
words

['go', 'until', 'jurong', 'point', 'crazy']

In [ ]:
vocab = sorted(list(set(words)))
word_to_index = {word: i for i, word in enumerate(vocab)}
print(f"Vocabulario: {word_to_index}")

Vocabulario: {'crazy': 0, 'go': 1, 'jurong': 2, 'point': 3, 'until': 4}


In [ ]:
word_indices = [word_to_index[word] for word in words]

In [ ]:
encoder = OneHotEncoder(categories=[range(len(vocab))], sparse_output=False)

In [ ]:
word_indices

[1, 4, 2, 3, 0]

In [ ]:
one_hot_vectors = encoder.fit_transform(np.array(word_indices).reshape(-1, 1))

print("\nRepresentación One-Hot de la oración:")
for word, vector in zip(words, one_hot_vectors):
    print(f"'{word}': {vector}")


Representación One-Hot de la oración:
'go': [0. 1. 0. 0. 0.]
'until': [0. 0. 0. 0. 1.]
'jurong': [0. 0. 1. 0. 0.]
'point': [0. 0. 0. 1. 0.]
'crazy': [1. 0. 0. 0. 0.]


# **Modelos Clásicos de Representación**

## **Bag of Words (BoW)**

El modelo **Bag of Words (BoW)** o "Bolsa de Palabras" simplifica la representación de un texto. En lugar de mantener un vector one-hot para cada palabra en una secuencia, simplemente contamos la frecuencia de cada palabra del vocabulario en el documento.

Imagina que "metemos todas las palabras de una oración en una bolsa", sin importar su orden, y luego contamos cuántas veces aparece cada una.

Por ejemplo, para la oración "él es él", con un vocabulario `{"él": 0, "es": 1}`, el vector BoW sería `[2, 1]`, porque "él" aparece dos veces y "es" una vez.

**Limitaciones de BoW:**
*   **Pierde el orden de las palabras:** Las frases "el perro muerde al hombre" y "el hombre muerde al perro" tienen la misma representación BoW, aunque su significado es completamente opuesto.
*   **No captura el significado:** Al igual que One-Hot Encoding, no entiende que "coche" y "auto" son sinónimos.
*   **Vectores dispersos y de alta dimensionalidad:** Sigue generando vectores muy grandes y llenos de ceros.

Ejemplo :
* Frase A → muy:2, rápido:1, pero:1, no:1, tanto:1 → Vector BoW: [2, 1, 1, 1, 1]

* Frase B → muy:2, rápido:1, pero:1, no:1, tanto:1 → Vector BoW: [2, 1, 1, 1, 1]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    "gato duerme",
    "perro ladra fuerte",
    "gato y perro juegan juntos en el parque"
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print("Vocabulario:", vectorizer.get_feature_names_out())
print("\nMatriz BoW:\n", X.toarray())


Vocabulario: ['duerme' 'el' 'en' 'fuerte' 'gato' 'juegan' 'juntos' 'ladra' 'parque'
 'perro']

Matriz BoW:
 [[1 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 1]
 [0 1 1 0 1 1 1 0 1 1]]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Dividir los datos (usando el texto limpiado)
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['cleaned_text'], df['label_num'], test_size=0.2, random_state=42
)

# Crear el vectorizador de BoW
bow_vectorizer = CountVectorizer()

# Aprender el vocabulario y transformar los datos de entrenamiento
X_train_bow = bow_vectorizer.fit_transform(X_train_text)

# Transformar los datos de prueba con el mismo vocabulario
X_test_bow = bow_vectorizer.transform(X_test_text)

print("Forma de la matriz BoW de entrenamiento:", X_train_bow.shape)

# Entrenar y evaluar el modelo
bow_model = LogisticRegression(max_iter=1000)
bow_model.fit(X_train_bow, y_train)
y_pred_bow = bow_model.predict(X_test_bow)

print("\nRendimiento con Bag of Words (BoW):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_bow):.4f}")
print(classification_report(y_test, y_pred_bow))

Forma de la matriz BoW de entrenamiento: (4459, 8324)

Rendimiento con Bag of Words (BoW):
Accuracy: 0.9821
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       954
           1       1.00      0.88      0.93       161

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



# 2.3.2 TF-IDF (con matemática)

Dado un término \( t \), documento \( d \) y corpus \( D \):

- **Frecuencia del término (TF):**
$$
TF(t,d) = \frac{\#\text{veces que } t \text{ aparece en } d}{\#\text{términos en } d}
$$

- **Frecuencia inversa de documentos (IDF):**
$$
IDF(t, D) = \log\left(\frac{|D|}{|\{d \in D : t \in d\}| + 1}\right)
$$

- **TF-IDF:**
$$
TF\text{-}IDF(t,d,D) = TF(t,d)\times IDF(t,D)
$$

**Intuición:** términos frecuentes en un documento pero raros en el corpus reciben más peso.  
**Limitaciones:** sigue ignorando orden y contexto.

Entrenamos **regresión logística** sobre TF-IDF.


**Ejemplo con un corpus pequeño:**

Corpus (3 documentos):

1. `"el gato duerme"`
2. `"el perro ladra fuerte"`
3. `"el gato y el perro juegan"`

---

#### 1. Vocabulario (palabras únicas en todo el corpus)

`["el", "gato", "duerme", "perro", "ladra", "fuerte", "y", "juegan"]` → 8 palabras.

---

#### 2. TF (frecuencia normalizada dentro de cada documento)

* Doc1: `"el gato duerme"` (3 términos)

  * TF(el)=1/3, TF(gato)=1/3, TF(duerme)=1/3, resto=0

* Doc2: `"el perro ladra fuerte"` (4 términos)

  * TF(el)=1/4, TF(perro)=1/4, TF(ladra)=1/4, TF(fuerte)=1/4, resto=0

* Doc3: `"el gato y el perro juegan"` (5 términos)

  * TF(el)=2/5, TF(gato)=1/5, TF(perro)=1/5, TF(y)=1/5, TF(juegan)=1/5, resto=0

---

#### 3. IDF (qué tan raro es un término en el corpus)

Fórmula:

$$
IDF(t, D) = \log\Big(\frac{|D|}{\#\text{docs con } t}+1\Big)
$$

* |D| = 3 documentos.
* `"el"` aparece en los 3 → IDF(el)= log(3/(3)+1) = log(2) ≈ 0.69 (poco informativa).
* `"gato"` aparece en 2 → IDF(gato)= log(3/2+1)= log(2.5) ≈ 0.92.
* `"duerme"` aparece en 1 → IDF(duerme)= log(3/1+1)= log(4) ≈ 1.39.
* `"ladra"` aparece en 1 → IDF(ladra)=1.39.
* `"fuerte"` aparece en 1 → IDF(fuerte)=1.39.
* `"perro"` aparece en 2 → IDF(perro)=0.92.
* `"y"` aparece en 1 → IDF(y)=1.39.
* `"juegan"` aparece en 1 → IDF(juegan)=1.39.

---

#### 4. TF-IDF (producto)

* Doc1 (`el gato duerme`):

  * el = 1/3 × 0.69 ≈ 0.23
  * gato = 1/3 × 0.92 ≈ 0.31
  * duerme = 1/3 × 1.39 ≈ 0.46
  * resto = 0

Vector ≈ `[0.23, 0.31, 0.46, 0, 0, 0, 0, 0]`

---

* Doc2 (`el perro ladra fuerte`):

  * el = 1/4 × 0.69 ≈ 0.17
  * perro = 1/4 × 0.92 ≈ 0.23
  * ladra = 1/4 × 1.39 ≈ 0.35
  * fuerte = 1/4 × 1.39 ≈ 0.35
  * resto = 0

Vector ≈ `[0.17, 0, 0, 0.23, 0.35, 0.35, 0, 0]`

---

* Doc3 (`el gato y el perro juegan`):

  * el = 2/5 × 0.69 ≈ 0.28
  * gato = 1/5 × 0.92 ≈ 0.18
  * perro = 1/5 × 0.92 ≈ 0.18
  * y = 1/5 × 1.39 ≈ 0.28
  * juegan = 1/5 × 1.39 ≈ 0.28

Vector ≈ `[0.28, 0.18, 0, 0.18, 0, 0, 0.28, 0.28]`

---

👉 Observa que:

* Palabras comunes como `"el"` tienen **peso bajo**.
* Palabras raras como `"duerme"`, `"fuerte"`, `"juegan"` tienen **peso alto**.
* El vector sigue siendo disperso, pero con valores **continuos**, no solo enteros como en BoW.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

corpus = [
    "el gato duerme",
    "el perro ladra fuerte",
    "el gato y el perro juegan"
]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Mostrar vocabulario y matriz TF-IDF
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print(df_tfidf.round(2))


   duerme    el  fuerte  gato  juegan  ladra  perro
0    0.72  0.43    0.00  0.55    0.00   0.00   0.00
1    0.00  0.35    0.58  0.00    0.00   0.58   0.44
2    0.00  0.63    0.00  0.40    0.53   0.00   0.40


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Crear el vectorizador de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Aprender el vocabulario y transformar los datos de entrenamiento
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)

# Transformar los datos de prueba
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

print("Forma de la matriz TF-IDF de entrenamiento:", X_train_tfidf.shape)

# Entrenar y evaluar el modelo
tfidf_model = LogisticRegression(max_iter=1000)
tfidf_model.fit(X_train_tfidf, y_train)
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

print("\nRendimiento con TF-IDF:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tfidf):.4f}")
print(classification_report(y_test, y_pred_tfidf))

Forma de la matriz TF-IDF de entrenamiento: (4459, 8324)

Rendimiento con TF-IDF:
Accuracy: 0.9614
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       954
           1       0.95      0.77      0.85       161

    accuracy                           0.96      1115
   macro avg       0.96      0.88      0.92      1115
weighted avg       0.96      0.96      0.96      1115



## Resumen
- **Features manuales**: rápidas pero limitadas semánticamente.  
- **BoW**: simple y efectivo; ignora orden y contexto.  
- **TF-IDF**: repondera términos útiles; sigue sin capturar contexto.




#Módulo 3 – Representaciones Distribuidas (Embeddings)

Los métodos como BoW y TF-IDF tratan las palabras como unidades aisladas. No pueden entender que "feliz" y "contento" son semánticamente similares. Los Word Embeddings o "incrustaciones de palabras" resuelven este problema representando las palabras como vectores densos en un espacio de baja dimensión (normalmente entre 50 y 300 dimensiones).

En este espacio vectorial, las palabras con significados similares se sitúan cerca unas de otras. Esto permite a los modelos capturar relaciones semánticas y sintácticas.
##Word Embeddings Estáticos

Los embeddings estáticos asignan un único vector a cada palabra, sin importar el contexto en el que aparezca. Los dos modelos más famosos son Word2Vec y GloVe.
##Word2Vec (CBOW y Skip-Gram)

Word2Vec es un modelo de red neuronal que aprende los embeddings a partir de un gran corpus de texto. Su principio es que "una palabra se conoce por las compañías que mantiene".

   * CBOW (Continuous Bag of Words): Predice una palabra a partir de las palabras de su contexto (ej. dadas ["El", "perro", "al", "vecino"], predecir "mordió").

   * Skip-Gram: Hace lo contrario. Dada una palabra, predice las palabras de su contexto (ej. dado "mordió", predecir ["El", "perro", "al", "vecino"]).

##GloVe (Global Vectors)

GloVe tiene un enfoque diferente. En lugar de mirar ventanas de contexto locales, construye una gran matriz de co-ocurrencia que cuenta cuántas veces cada par de palabras aparecen juntas en todo el corpus. Luego, factoriza esta matriz para obtener los vectores de palabras.

Limitaciones de los embeddings estáticos:

   * Polisemia: No pueden manejar palabras con múltiples significados. La palabra "banco" tendrá el mismo vector si se refiere a una institución financiera o a un asiento.

   * Contexto estático: El vector de una palabra es siempre el mismo, sin importar la oración.

   * Palabras fuera de vocabulario (OOV): Si una palabra no estaba en el texto de entrenamiento, no tendrá un vector.

Para usar estos embeddings en un clasificador de documentos, necesitamos una estrategia para agregar los vectores de todas las palabras de un mensaje en un solo vector que represente todo el mensaje. Un método común y simple es promediar los vectores de todas las palabras.

In [ ]:

import gensim.downloader as api

# Cargar un modelo GloVe pre-entrenado (vectores de 50 dimensiones)
# Esto descargará el modelo la primera vez que se ejecute (aprox. 128MB)
glove_model = api.load("glove-wiki-gigaword-50")
embedding_dim = glove_model.vector_size

def document_vector(doc):
    """Crea un vector de documento promediando los embeddings de sus palabras."""
    # Quitar palabras que no están en el vocabulario del modelo GloVe
    doc_words = [word for word in doc.split() if word in glove_model]
    if not doc_words:
        return np.zeros(embedding_dim)

    # Promediar los vectores de las palabras
    return np.mean(glove_model[doc_words], axis=0)

# Aplicar la función a nuestros datos
X_train_glove = np.array([document_vector(doc) for doc in X_train_text])
X_test_glove = np.array([document_vector(doc) for doc in X_test_text])

print("Forma de la matriz de embeddings de entrenamiento:", X_train_glove.shape)

# Entrenar y evaluar el modelo
glove_model_clf = LogisticRegression(max_iter=1000)
glove_model_clf.fit(X_train_glove, y_train)
y_pred_glove = glove_model_clf.predict(X_test_glove)

print("\nRendimiento con Word Embeddings (GloVe promediado):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_glove):.4f}")
print(classification_report(y_test, y_pred_glove))

[==================================================] 100.0% 66.0/66.0MB downloaded
Forma de la matriz de embeddings de entrenamiento: (4459, 50)

Rendimiento con Word Embeddings (GloVe promediado):
Accuracy: 0.9220
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       954
           1       0.81      0.60      0.69       161

    accuracy                           0.92      1115
   macro avg       0.87      0.79      0.82      1115
weighted avg       0.92      0.92      0.92      1115



#Embeddings Contextuales (BERT)


Los embeddings contextuales son la solución al problema de la polisemia. En lugar de tener un vector fijo para cada palabra, estos modelos generan un vector dinámico que depende del contexto de la oración en la que se encuentra.

El modelo más influyente en esta área es BERT (Bidirectional Encoder Representations from Transformers). BERT procesa la oración completa de una vez, considerando tanto las palabras a la izquierda como a la derecha para generar la representación de cada palabra.

Tokenización por Subpalabras con WordPiece:
Los modelos como BERT no trabajan con palabras completas. Usan una técnica de tokenización por subpalabras como WordPiece. Esto les permite:

   * Manejar palabras raras o desconocidas: Una palabra desconocida como "playing" puede descomponerse en subpalabras conocidas como "play" y "##ing". El prefijo ## indica que es la continuación de una palabra.

   * Reducir el tamaño del vocabulario: Se necesita un vocabulario mucho más pequeño para representar un idioma completo.

   * Capturar información morfológica: El modelo puede aprender que el sufijo ##ndo tiene un significado relacionado con acciones en progreso.

Para la clasificación de textos, BERT utiliza un token especial llamado [CLS] (Classification) al inicio de cada secuencia. La idea es que el embedding de salida correspondiente a este token actúe como una representación agregada de toda la secuencia, lista para ser usada por un clasificador.

In [ ]:
# ======================
# BERT embeddings (CLS) con transformers

import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tok = BertTokenizer.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased").to(device)
bert.eval()

def bert_cls_embedding(texts, batch_size=32, max_length=128):
    embs = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        enc = tok(batch, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
        with torch.no_grad():
            for k in enc:
                enc[k] = enc[k].to(device)
            out = bert(**enc).last_hidden_state  # [B, L, H]
            cls = out[:, 0, :].cpu().numpy()     # [CLS]
            embs.append(cls)
    return np.vstack(embs)

X_bert = bert_cls_embedding(df["text"].tolist(), batch_size=32, max_length=128)

Xtr_b, Xte_b, ytr_b, yte_b = train_test_split(X_bert, y, test_size=0.2, random_state=123, stratify=y)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 175/175 [14:54<00:00,  5.11s/it]

Accuracy (BERT [CLS] embeddings): 0.9946


In [ ]:
clf_bert = LogisticRegression(max_iter=2000)
clf_bert.fit(Xtr_b, ytr_b)
pred_b = clf_bert.predict(Xte_b)
acc_b = accuracy_score(yte_b, pred_b)
print("\nRendimiento con Word Embeddings (GloVe promediado):")
print(f"Accuracy: {accuracy_score(yte_b, pred_b):.4f}")
print(classification_report(yte_b, pred_b))
print("Accuracy (BERT [CLS] embeddings):", round(acc_b, 4))


Rendimiento con Word Embeddings (GloVe promediado):
Accuracy: 0.9946
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       966
           1       0.99      0.97      0.98       149

    accuracy                           0.99      1115
   macro avg       0.99      0.98      0.99      1115
weighted avg       0.99      0.99      0.99      1115

Accuracy (BERT [CLS] embeddings): 0.9946


In [ ]:
from transformers import BertTokenizer
tokenizer_wp = BertTokenizer.from_pretrained('bert-base-uncased')

word = "playing restarting unbelievable SMS Spammers!!!"
toks = tokenizer_wp.tokenize(word)
ids = tokenizer_wp.convert_tokens_to_ids(toks)
print(f"Palabra: {word}\nTokens: {toks}\nIDs: {ids}\n")


Palabra: playing restarting unbelievable SMS Spammers!!!
Tokens: ['playing', 'restart', '##ing', 'unbelievable', 'sms', 'spa', '##mmer', '##s', '!', '!', '!']
IDs: [2652, 23818, 2075, 23653, 22434, 12403, 15810, 2015, 999, 999, 999]

